In [1]:
using NBInclude
@nbinclude("dcj_algo.ipynb")

SRC (.,a:t)(a:h,.)(.,b:t)(b:h,.)(.,c:t)(c:h,.)(.,d:t)(d:h,.) --> TARGET (a:t,b:h)(b:t,c:h)(c:t,d:h)(d:t,a:h)
DCJ :: (.,a:t)(b:h,.) --> (a:t,b:h)(.,.)
DCJ :: (.,b:t)(c:h,.) --> (b:t,c:h)(.,.)
DCJ :: (.,c:t)(d:h,.) --> (c:t,d:h)(.,.)
DCJ :: (.,d:t)(a:h,.) --> (d:t,a:h)(.,.)

#ops = 4
----------------------------------------


4

In [2]:
function first_n_letters_no_limits(n::Int)
    alphabet = 'a':'z'  # Define the alphabet range
    letters = []
    
    # Iterate from 1 to n
    for i in 1:n
        idx = i - 1
        letter = ""
        
        # Calculate each place value
        while idx >= 0
            place_value = idx % 26 + 1
            letter = string(alphabet[place_value], letter)
            idx = idx ÷ 26 - 1
        end
        
        push!(letters, letter)
    end
    
    return letters
end

# println(letters_up_to_n(26))  # Output: ["A", "B", ..., "Z"]
# println(letters_up_to_n(27))  # Output: ["AA"]
# println(letters_up_to_n(53))  # Output: ["BA", "BB", ..., "BZ"]
# println(letters_up_to_n(54))  # Output: ["CA"]
# println(letters_up_to_n(703)) # Output: ["ZZ"]

first_n_letters_no_limits (generic function with 1 method)

In [3]:
# generates target genomes that contain the first n letters and adds x duplicate letters 
# x >= 1
function generate_target_with_xdup(n::Int, x::Int)
    # target 
    alphabet = first_n_letters_no_limits(n)
    genome = copy(alphabet)
    
    for i in 1:x
        rand_dup_gene = alphabet[rand(1:length(alphabet))]
        rand_idx = rand(1:length(genome)+1)
        insert!(genome, rand_idx, rand_dup_gene)
    end 

    return join(genome, "'")  # one chrom 
end 

# target = generate_target_with_xdup(5, 2)
# print(target)

generate_target_with_xdup (generic function with 1 method)

In [4]:
function add_gene_counts(input_string::String)  
    genes = split(input_string, "'")

    gene_count = Dict{String, Int}()
    result = IOBuffer()

    for gene in genes
        if gene == '.' || gene == ',' 
            print(result, gene) 
            continue 
        end 
        if !haskey(gene_count, gene)
            gene_count[gene] = 0
        end
        gene_count[gene] += 1
        print(result, gene, gene_count[gene])
    end

    return String(take!(result))
end

# input_string = "he'll'o"
# result = add_letter_counts(input_string)
# println(result)  # Output: "he1ll1o1"

add_gene_counts (generic function with 1 method)

In [7]:
tar = generate_target_with_xdup(27, 3)
tar_wcounts = add_gene_counts(tar)
println(tar) 
println(tar_wcounts)

i'a'b'c'v'd'e'f'g'h'i'j'k'l'm'n'o'p'q'r's't'u'v'w'x'y'z'aa'i
i1a1b1c1v1d1e1f1g1h1i2j1k1l1m1n1o1p1q1r1s1t1u1v2w1x1y1z1aa1i3


In [10]:
# function apply_dcjop(adj_list::Vector{Adjacency})

# end 

function apply_x_dcjops(target::String, x::Int)
    # turn target genome into list of adjacencies
    id_counter = Ref{Int}(1)
    id_to_str = Dict{Int, String}()
    str_to_id = Dict{String, Int}()

    target_genome = string_to_genome(target, id_counter, id_to_str, str_to_id, true)
    show(target_genome)
    adj_list = genome_to_adj_list(target_genome)     
    show(adj_list)  
    # for i in 1:x
    #     # apply dcj op to list of adjacencies 
    # end 

    # convert adj back to genome 

    # return genome 
    return 
end 


apply_x_dcjops(tar_wcounts, 3)

iabcvdefghijklmnopqrstuvwxyzaai(i:h,a:t)(a:h,b:t)(b:h,c:t)(c:h,v:t)(v:h,d:t)(d:h,e:t)(e:h,f:t)(f:h,g:t)(g:h,h:t)(h:h,i:t)(i:h,j:t)(j:h,k:t)(k:h,l:t)(l:h,m:t)(m:h,n:t)(n:h,o:t)(o:h,p:t)(p:h,q:t)(q:h,r:t)(r:h,s:t)(s:h,t:t)(t:h,u:t)(u:h,v:t)(v:h,w:t)(w:h,x:t)(x:h,y:t)(y:h,z:t)(z:h,aa:t)(aa:h,i:t)(i:h,i:t)